In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l', save_generated_data=False, additional_letter_attributes=False, reverse_inputs=False)
data.generate_data('TEST_train',
                   'TEST_test',
                   'TEST_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
# for el in test2[0][0]:
# #     print(el)
#     print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

10
(16, 23, 36)
(16, 140)


In [12]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.x_other_features_train[pos])
print(data.decode_y(data.y_train[pos]))

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
ašjenlautkajan
Agsnpn-
Agsnpn-
[1 1 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
[2, 5]


In [11]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20



# conv_input_shape=(23, 36)
conv_input_shape=(23, 30)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [12]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch),
                              verbose=2)

Epoch 1/800
17s - loss: 0.2303 - actual_accuracy: 0.2434 - val_loss: 0.1781 - val_actual_accuracy: 0.4996
Epoch 2/800
17s - loss: 0.1635 - actual_accuracy: 0.4785 - val_loss: 0.1507 - val_actual_accuracy: 0.5832
Epoch 3/800
18s - loss: 0.1412 - actual_accuracy: 0.5678 - val_loss: 0.1315 - val_actual_accuracy: 0.6576
Epoch 4/800
25s - loss: 0.1237 - actual_accuracy: 0.6471 - val_loss: 0.1162 - val_actual_accuracy: 0.7205
Epoch 5/800
26s - loss: 0.1114 - actual_accuracy: 0.7042 - val_loss: 0.1137 - val_actual_accuracy: 0.7327
Epoch 6/800
19s - loss: 0.1027 - actual_accuracy: 0.7355 - val_loss: 0.0981 - val_actual_accuracy: 0.7692
Epoch 7/800
25s - loss: 0.0943 - actual_accuracy: 0.7641 - val_loss: 0.0947 - val_actual_accuracy: 0.7910
Epoch 8/800
19s - loss: 0.0914 - actual_accuracy: 0.7734 - val_loss: 0.0921 - val_actual_accuracy: 0.7906
Epoch 9/800
20s - loss: 0.0844 - actual_accuracy: 0.7977 - val_loss: 0.0948 - val_actual_accuracy: 0.7998
Epoch 10/800
21s - loss: 0.0808 - actual_accur

24s - loss: 0.0180 - actual_accuracy: 0.9612 - val_loss: 0.1320 - val_actual_accuracy: 0.8570
Epoch 79/800
22s - loss: 0.0181 - actual_accuracy: 0.9604 - val_loss: 0.1114 - val_actual_accuracy: 0.8692
Epoch 80/800
22s - loss: 0.0187 - actual_accuracy: 0.9605 - val_loss: 0.1179 - val_actual_accuracy: 0.8589
Epoch 81/800
22s - loss: 0.0172 - actual_accuracy: 0.9618 - val_loss: 0.1144 - val_actual_accuracy: 0.8677
Epoch 82/800
22s - loss: 0.0176 - actual_accuracy: 0.9627 - val_loss: 0.1130 - val_actual_accuracy: 0.8666
Epoch 83/800
23s - loss: 0.0174 - actual_accuracy: 0.9625 - val_loss: 0.0963 - val_actual_accuracy: 0.8811
Epoch 84/800
22s - loss: 0.0173 - actual_accuracy: 0.9638 - val_loss: 0.1023 - val_actual_accuracy: 0.8804
Epoch 85/800
22s - loss: 0.0166 - actual_accuracy: 0.9653 - val_loss: 0.1206 - val_actual_accuracy: 0.8689
Epoch 86/800
22s - loss: 0.0162 - actual_accuracy: 0.9642 - val_loss: 0.1037 - val_actual_accuracy: 0.8635
Epoch 87/800
22s - loss: 0.0166 - actual_accuracy:

23s - loss: 0.0100 - actual_accuracy: 0.9786 - val_loss: 0.1259 - val_actual_accuracy: 0.8646
Epoch 156/800
23s - loss: 0.0101 - actual_accuracy: 0.9784 - val_loss: 0.1164 - val_actual_accuracy: 0.8877
Epoch 157/800
23s - loss: 0.0090 - actual_accuracy: 0.9802 - val_loss: 0.1255 - val_actual_accuracy: 0.8838
Epoch 158/800
21s - loss: 0.0097 - actual_accuracy: 0.9797 - val_loss: 0.1203 - val_actual_accuracy: 0.8842
Epoch 159/800
21s - loss: 0.0103 - actual_accuracy: 0.9786 - val_loss: 0.1394 - val_actual_accuracy: 0.8646
Epoch 160/800
22s - loss: 0.0096 - actual_accuracy: 0.9808 - val_loss: 0.1036 - val_actual_accuracy: 0.8915
Epoch 161/800
24s - loss: 0.0098 - actual_accuracy: 0.9796 - val_loss: 0.1084 - val_actual_accuracy: 0.8850
Epoch 162/800
23s - loss: 0.0098 - actual_accuracy: 0.9798 - val_loss: 0.1252 - val_actual_accuracy: 0.8681
Epoch 163/800
23s - loss: 0.0091 - actual_accuracy: 0.9812 - val_loss: 0.1154 - val_actual_accuracy: 0.8808
Epoch 164/800
22s - loss: 0.0090 - actual_

23s - loss: 0.0071 - actual_accuracy: 0.9850 - val_loss: 0.1450 - val_actual_accuracy: 0.8728
Epoch 232/800
23s - loss: 0.0070 - actual_accuracy: 0.9858 - val_loss: 0.1387 - val_actual_accuracy: 0.8677
Epoch 233/800
24s - loss: 0.0063 - actual_accuracy: 0.9867 - val_loss: 0.1343 - val_actual_accuracy: 0.8788
Epoch 234/800
23s - loss: 0.0075 - actual_accuracy: 0.9861 - val_loss: 0.1290 - val_actual_accuracy: 0.8689
Epoch 235/800
23s - loss: 0.0072 - actual_accuracy: 0.9862 - val_loss: 0.1140 - val_actual_accuracy: 0.8831
Epoch 236/800
23s - loss: 0.0070 - actual_accuracy: 0.9857 - val_loss: 0.1296 - val_actual_accuracy: 0.8785
Epoch 237/800
23s - loss: 0.0061 - actual_accuracy: 0.9875 - val_loss: 0.1315 - val_actual_accuracy: 0.8777
Epoch 238/800
24s - loss: 0.0069 - actual_accuracy: 0.9869 - val_loss: 0.1071 - val_actual_accuracy: 0.8957
Epoch 239/800
23s - loss: 0.0068 - actual_accuracy: 0.9861 - val_loss: 0.1315 - val_actual_accuracy: 0.8762
Epoch 240/800
23s - loss: 0.0080 - actual_

23s - loss: 0.0057 - actual_accuracy: 0.9890 - val_loss: 0.1292 - val_actual_accuracy: 0.8861
Epoch 308/800
23s - loss: 0.0055 - actual_accuracy: 0.9889 - val_loss: 0.1505 - val_actual_accuracy: 0.8670
Epoch 309/800
25s - loss: 0.0051 - actual_accuracy: 0.9899 - val_loss: 0.1463 - val_actual_accuracy: 0.8700
Epoch 310/800
23s - loss: 0.0056 - actual_accuracy: 0.9887 - val_loss: 0.1196 - val_actual_accuracy: 0.8923
Epoch 311/800
23s - loss: 0.0053 - actual_accuracy: 0.9895 - val_loss: 0.1385 - val_actual_accuracy: 0.8873
Epoch 312/800
22s - loss: 0.0058 - actual_accuracy: 0.9883 - val_loss: 0.1407 - val_actual_accuracy: 0.8742
Epoch 313/800
23s - loss: 0.0059 - actual_accuracy: 0.9889 - val_loss: 0.1292 - val_actual_accuracy: 0.8846
Epoch 314/800
24s - loss: 0.0062 - actual_accuracy: 0.9886 - val_loss: 0.1517 - val_actual_accuracy: 0.8765
Epoch 315/800
23s - loss: 0.0058 - actual_accuracy: 0.9884 - val_loss: 0.1307 - val_actual_accuracy: 0.8915
Epoch 316/800
23s - loss: 0.0053 - actual_

...

25s - loss: 0.0044 - actual_accuracy: 0.9916 - val_loss: 0.1511 - val_actual_accuracy: 0.8854
Epoch 460/800
23s - loss: 0.0046 - actual_accuracy: 0.9914 - val_loss: 0.1374 - val_actual_accuracy: 0.8846
Epoch 461/800
24s - loss: 0.0054 - actual_accuracy: 0.9888 - val_loss: 0.1381 - val_actual_accuracy: 0.8831
Epoch 462/800
24s - loss: 0.0043 - actual_accuracy: 0.9915 - val_loss: 0.1511 - val_actual_accuracy: 0.8766
Epoch 463/800
24s - loss: 0.0037 - actual_accuracy: 0.9924 - val_loss: 0.1466 - val_actual_accuracy: 0.8700
Epoch 464/800
25s - loss: 0.0047 - actual_accuracy: 0.9905 - val_loss: 0.1291 - val_actual_accuracy: 0.8911
Epoch 465/800
23s - loss: 0.0046 - actual_accuracy: 0.9901 - val_loss: 0.1424 - val_actual_accuracy: 0.8900
Epoch 466/800
23s - loss: 0.0044 - actual_accuracy: 0.9904 - val_loss: 0.1463 - val_actual_accuracy: 0.8846
Epoch 467/800
23s - loss: 0.0049 - actual_accuracy: 0.9908 - val_loss: 0.1742 - val_actual_accuracy: 0.8715
Epoch 468/800
23s - loss: 0.0044 - actual_

23s - loss: 0.0044 - actual_accuracy: 0.9909 - val_loss: 0.1524 - val_actual_accuracy: 0.8758
Epoch 536/800
24s - loss: 0.0046 - actual_accuracy: 0.9916 - val_loss: 0.1359 - val_actual_accuracy: 0.8919
Epoch 537/800
24s - loss: 0.0037 - actual_accuracy: 0.9928 - val_loss: 0.1486 - val_actual_accuracy: 0.8811
Epoch 538/800
25s - loss: 0.0038 - actual_accuracy: 0.9922 - val_loss: 0.1494 - val_actual_accuracy: 0.8777
Epoch 539/800
24s - loss: 0.0042 - actual_accuracy: 0.9914 - val_loss: 0.1869 - val_actual_accuracy: 0.8686
Epoch 540/800
24s - loss: 0.0046 - actual_accuracy: 0.9910 - val_loss: 0.1701 - val_actual_accuracy: 0.8643
Epoch 541/800
23s - loss: 0.0044 - actual_accuracy: 0.9919 - val_loss: 0.1678 - val_actual_accuracy: 0.8669
Epoch 542/800
23s - loss: 0.0037 - actual_accuracy: 0.9923 - val_loss: 0.1605 - val_actual_accuracy: 0.8938
Epoch 543/800
25s - loss: 0.0041 - actual_accuracy: 0.9920 - val_loss: 0.1680 - val_actual_accuracy: 0.8727
Epoch 544/800
24s - loss: 0.0046 - actual_

26s - loss: 0.0038 - actual_accuracy: 0.9925 - val_loss: 0.1737 - val_actual_accuracy: 0.8735
Epoch 612/800
24s - loss: 0.0039 - actual_accuracy: 0.9928 - val_loss: 0.1743 - val_actual_accuracy: 0.8758
Epoch 613/800
24s - loss: 0.0036 - actual_accuracy: 0.9934 - val_loss: 0.1455 - val_actual_accuracy: 0.8888
Epoch 614/800
23s - loss: 0.0043 - actual_accuracy: 0.9913 - val_loss: 0.1474 - val_actual_accuracy: 0.8926
Epoch 615/800
24s - loss: 0.0038 - actual_accuracy: 0.9920 - val_loss: 0.1536 - val_actual_accuracy: 0.8827
Epoch 616/800
25s - loss: 0.0033 - actual_accuracy: 0.9931 - val_loss: 0.1889 - val_actual_accuracy: 0.8682
Epoch 617/800
24s - loss: 0.0032 - actual_accuracy: 0.9942 - val_loss: 0.1786 - val_actual_accuracy: 0.8627
Epoch 618/800
24s - loss: 0.0039 - actual_accuracy: 0.9925 - val_loss: 0.1688 - val_actual_accuracy: 0.8892
Epoch 619/800
24s - loss: 0.0044 - actual_accuracy: 0.9914 - val_loss: 0.1331 - val_actual_accuracy: 0.8873
Epoch 620/800
24s - loss: 0.0041 - actual_

24s - loss: 0.0033 - actual_accuracy: 0.9938 - val_loss: 0.1614 - val_actual_accuracy: 0.8792
Epoch 688/800
24s - loss: 0.0029 - actual_accuracy: 0.9942 - val_loss: 0.1797 - val_actual_accuracy: 0.8785
Epoch 689/800
26s - loss: 0.0036 - actual_accuracy: 0.9934 - val_loss: 0.1912 - val_actual_accuracy: 0.8719
Epoch 690/800
24s - loss: 0.0047 - actual_accuracy: 0.9908 - val_loss: 0.1372 - val_actual_accuracy: 0.8804
Epoch 691/800
24s - loss: 0.0037 - actual_accuracy: 0.9926 - val_loss: 0.1459 - val_actual_accuracy: 0.8919
Epoch 692/800
24s - loss: 0.0041 - actual_accuracy: 0.9927 - val_loss: 0.1494 - val_actual_accuracy: 0.8865
Epoch 693/800
24s - loss: 0.0033 - actual_accuracy: 0.9937 - val_loss: 0.1987 - val_actual_accuracy: 0.8559
Epoch 694/800
24s - loss: 0.0041 - actual_accuracy: 0.9926 - val_loss: 0.1793 - val_actual_accuracy: 0.8719
Epoch 695/800
24s - loss: 0.0039 - actual_accuracy: 0.9926 - val_loss: 0.1641 - val_actual_accuracy: 0.8880
Epoch 696/800
24s - loss: 0.0033 - actual_

24s - loss: 0.0031 - actual_accuracy: 0.9933 - val_loss: 0.1700 - val_actual_accuracy: 0.8869
Epoch 764/800
24s - loss: 0.0032 - actual_accuracy: 0.9938 - val_loss: 0.1491 - val_actual_accuracy: 0.8957
Epoch 765/800
24s - loss: 0.0036 - actual_accuracy: 0.9939 - val_loss: 0.1805 - val_actual_accuracy: 0.8746
Epoch 766/800
26s - loss: 0.0033 - actual_accuracy: 0.9941 - val_loss: 0.1668 - val_actual_accuracy: 0.8785
Epoch 767/800
24s - loss: 0.0035 - actual_accuracy: 0.9939 - val_loss: 0.1634 - val_actual_accuracy: 0.8857
Epoch 768/800
24s - loss: 0.0031 - actual_accuracy: 0.9946 - val_loss: 0.1641 - val_actual_accuracy: 0.8884
Epoch 769/800
24s - loss: 0.0034 - actual_accuracy: 0.9934 - val_loss: 0.1600 - val_actual_accuracy: 0.8869
Epoch 770/800
24s - loss: 0.0031 - actual_accuracy: 0.9939 - val_loss: 0.1776 - val_actual_accuracy: 0.8732
Epoch 771/800
26s - loss: 0.0034 - actual_accuracy: 0.9932 - val_loss: 0.1700 - val_actual_accuracy: 0.8727
Epoch 772/800
24s - loss: 0.0032 - actual_

In [10]:
name = '40_epoch'
# model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [7]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()